<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/logo.png?raw=true'/></center>

<h1 align = 'center'> Trabalho de Circuitos Elétricos - II<h1/>
<h2 align = 'center'> Simulação de Sistemas Elétricos por Quadripolos <h2\>

Este projeto se refere ao estudo sobre dimensionamento de filtros ativos, no qual o mesmo está salvo e disponível para consulta no seguinte [repositório](https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles).

O projeto foi realizado pelos seguinte alunos de Engenharia Elétrica da UFCG:

- Breno Henrique Martins Silva;
- Caíque Moraes de Lima Mangabeira;
- Emerson Matheus Lima da Silva
- Marcos Eduardo Araújo  
- Marcus Vinicius Almeida Filho

---

Utilizando dos conteúdos repassados nas aulas e também nos livros textos que se retratam sobre teoria de quadripolos, iremos utilizar a modelagem de dos componentes da seguinte forma:

### Linha de Transmissão de modelo $\pi$

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/linha.png?raw=true'/></center>

Linhas de transmissão são estruturas físicas utilizadas para transportar energia elétrica de uma fonte (geradores) até os consumidores finais (residências, indústrias, etc.) através de longas distâncias. As principais carácteristicas das Linhas de Transmissão, que estão distribuidas por toda a linha, são a sua Impedância Característica $Z = R + jX_L$ que influência diretamente em como a energia é transmitida e distribuída ao longo da linha, e sua admitância $Y = jX_C$ a admitância é um fator importante que modela o efeito da capacitância e das perdas dielétricas em linhas de transmissão. Seu impacto é notado especialmente em linhas longas, onde contribui para a corrente reativa e pode afetar a regulação de tensão e o fluxo de potência em sistemas de transmissão e distribuição de energia elétrica. 

As Linhas de Transmissão de modelo $\pi$ possuem um comprimento de 80 a 250 quilômetros, e para que possamos trabalhar com este modelo devemos concentrar seus parâmetro em uma equação matricial, no qual a tensão e correte no final e início da linhas de transmissão possam ser determinadas, assim utilizamos a seguinte matriz para representar o modelo do tipo $\pi$ para linhas de transmissão:

$$
\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 + \frac{ZY}{2} & Z\\ 
Y+\frac{ZY^{2}}{4} & 1 + \frac{ZY}{2}
\end{bmatrix} \cdot \begin{bmatrix}
V_2\\ 
I_2
\end{bmatrix}
$$
---

### Transformador

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/transformador.png?raw=true'/></center>

Iremos utilizar um transformador de um sistema elétrico de potência com o modelo como mostrado acima, e assim como nas Linhas de Transmissão ele também possui uma matriz que representa a relação entre tensão e corrente nos seus terminais, utilizando o conceito de um transformador ideal temos que a sua matriz de transformação é da seguinte forma:

$$
\begin{bmatrix}
n & 0\\
0 & \frac{1}{n}
\end{bmatrix}
$$

Utilizando a matriz de transformação de um Transformador Ideal e seguinte o modelo 

$$
\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
n(1+YZ_1) &\frac{1}{n}(Z_1+Z_2+YZ_1Z_2) \\ 
nY & \frac{1}{n}(1+YZ_2)
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
I_2
\end{bmatrix}
$$

In [ ]:
# Importando as Bibliotecas
import numpy as np

In [ ]:
# Declaração de todas as funções:

def MatrixSeriesImpedance(Z):                                   # Retorna a matriz referente a uma impedância em série.
    return np.array([1, Z], [0, 1])

def MarixShuntAdimitance(Y_C):                                  # Retorna a matriz referente a carga em derivação.
    return np.array([1, 0], [Y, 1])